In [41]:
import pandas_datareader.data as web
import pandas as pd
import datetime
from decimal import Decimal
from dateutil import relativedelta
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, resample_apply
import importlib
import plotly.express as px

#Change me!
start = datetime.datetime.today() + relativedelta.relativedelta(years=-5)
split_unit = "6M"
modulename = "BacktestingStrategies.Strategy_SMAandRSI"
classname = "btst.EntryRSIandExitSMA_WithShortPosition"
end = datetime.date.today()

data = web.DataReader('1357.T', 'yahoo', start, end)  #NEXT FUNDS 日経平均ダブルインバース・インデックス連動型上場投信
data = data.astype("double")

initial = 1000_000
fname = f"html/plot-"

In [42]:
#日付データを検証期間単位ごとにSplit
#https://kakakakakku.hatenablog.com/entry/2021/05/24/002542
grouped_data = data.groupby(pd.Grouper(freq = split_unit))
dfs = [group for _, group in grouped_data]

In [43]:
#検証期間単位ごとにバックテスト＆結果を蓄積
#親ディレクトリ経由で呼び出し   # https://qiita.com/yokohama4580/items/466a483ae022d264c8ee
import os
import sys
sys.path.append(os.pardir)

btst = importlib.import_module(modulename)
# from BacktestingStrategies import Strategy_RsiOscillator as btst
importlib.reload(btst)
import warnings
warnings.simplefilter('ignore')

returns = []

for monthly_data in dfs:
    # バックテストを設定
    bt = Backtest(
        monthly_data, # チャートデータ
        eval(classname), # 売買戦略
        cash=initial, # 最初の所持金
        commission=0.00495, # 取引手数料
        margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
        trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
        exclusive_orders=True #自動でポジションをクローズ(オープン)
    )

    output = bt.run() # バックテスト実行
    returns.append(output)

In [44]:
import statistics

df = pd.DataFrame(returns)

#最低と最大の期間のBacktestingを表示＆実行
lines = "１期間単位: " + split_unit + "\n"
lines += "テスト期間: " + str(len(returns)) + "\n"
res = round(statistics.mean(df["Return [%]"]), 4)

#リターンが最小のものの期間を取得
min_return = df[df["# Trades"] != 0]["Return [%]"].min()
min_period_start = pd.to_datetime(df[df["Return [%]"] == min_return]["Start"]).values[0]
min_period_end = pd.to_datetime(df[df["Return [%]"] == min_return]["End"]).values[0]
df_min = data[min_period_start : min_period_end]

#リターンが2番目、3番目に小さいものの期間を取得
min_return_3 = df[df["# Trades"] != 0]["Return [%]"].nsmallest(3)
min2_period_start = pd.to_datetime(df[df["Return [%]"] == min_return_3.values[1]]["Start"]).values[0]
min2_period_end = pd.to_datetime(df[df["Return [%]"] == min_return_3.values[1]]["End"]).values[0]
min3_period_start = pd.to_datetime(df[df["Return [%]"] == min_return_3.values[2]]["Start"]).values[0]
min3_period_end = pd.to_datetime(df[df["Return [%]"] == min_return_3.values[2]]["End"]).values[0]
df_min2 = data[min2_period_start : min2_period_end]
df_min3 = data[min3_period_start : min3_period_end]

#リターンが最大のものの期間を取得
max_return = df[df["# Trades"] != 0]["Return [%]"].max()
max_period_start = pd.to_datetime(df[df["Return [%]"] == max_return]["Start"]).values[0]
max_period_end = pd.to_datetime(df[df["Return [%]"] == max_return]["End"]).values[0]
df_max = data[max_period_start : max_period_end]

lines += "平均Return [%]: " + str(res) + "\n"
lines += "最大Return [%]: " + str(max_return) + "\n"
lines += "最大Return期間: " + str(max_period_start) + " - " + str(max_period_end) + "\n"
lines += "最低Return [%]: " + str(min_return) + "\n"
lines += "最低Return期間: " + str(min_period_start) + " - " + str(min_period_end) 

print(lines)



１期間単位: 12M
テスト期間: 6
平均Return [%]: 13.4567
最大Return [%]: 41.70668510499997
最大Return期間: 2018-10-01T00:00:00.000000000 - 2019-09-30T00:00:00.000000000
最低Return [%]: -17.220470880000025
最低Return期間: 2019-10-01T00:00:00.000000000 - 2020-09-30T00:00:00.000000000


In [45]:
#取引ない月込み
fig = px.box(df["Return [%]"], points = "all")
fig.update_layout(
    xaxis_title = "Strategy",
    yaxis_title = "Return [%]"
)
fig.show()

In [46]:
#最低と最高の期間のバックテストを表示

print("----------WORST----------")
bt = Backtest(
    df_min, # チャートデータ
    eval(classname), # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True, #自動でポジションをクローズ(オープン)
)
output = bt.run() # バックテスト実行
print(output) # 実行結果(データ)
display(output._trades)
bt.plot(filename = fname + str(output._strategy) + "_min.html") # 実行結果（グラフ）

print("----------MIN2----------")
bt = Backtest(
    df_min2, # チャートデータ
    eval(classname), # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True, #自動でポジションをクローズ(オープン)
)
output = bt.run() # バックテスト実行
print(output) # 実行結果(データ)
display(output._trades)
bt.plot(filename = fname + str(output._strategy) + "_min2.html") # 実行結果（グラフ）

print("----------MIN3----------")
bt = Backtest(
    df_min3, # チャートデータ
    eval(classname), # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True, #自動でポジションをクローズ(オープン)
)
output = bt.run() # バックテスト実行
print(output) # 実行結果(データ)
display(output._trades)
bt.plot(filename = fname + str(output._strategy) + "_min3.html") # 実行結果（グラフ）

print("----------BEST----------")
bt = Backtest(
    df_max, # チャートデータ
    eval(classname), # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True, #自動でポジションをクローズ(オープン)
)
output = bt.run() # バックテスト実行
print(output) # 実行結果(データ)
display(output._trades)
bt.plot(filename = fname + str(output._strategy) + "_max.html") # 実行結果（グラフ）

----------WORST----------
Start                     2019-10-01 00:00:00
End                       2020-09-30 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                   61.570248
Equity Final [$]                  827795.2912
Equity Peak [$]                   1056851.032
Return [%]                         -17.220471
Buy & Hold Return [%]               -32.44529
Return (Ann.) [%]                  -17.864418
Volatility (Ann.) [%]               81.083296
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -64.744603
Avg. Drawdown [%]                  -22.717582
Max. Drawdown Duration      231 days 00:00:00
Avg. Drawdown Duration       81 days 00:00:00
# Trades                                    7
Win Rate [%]                        28.571429
Best Trade [%]                      14.317888
Worst Trade [%]                     -9.180638
Avg. Tra

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-1104,80,97,905.49550,976.00,-77836.96800,-0.077863,2020-01-31,2020-02-27,27 days
1,-879,98,141,1048.78270,1039.00,8598.99330,0.009328,2020-02-28,2020-05-01,63 days
2,938,145,155,991.88565,931.00,-57110.73970,-0.061384,2020-05-12,2020-05-26,14 days
3,961,156,157,908.47480,855.95,-50476.33280,-0.057816,2020-05-27,2020-05-28,1 days
4,953,158,161,863.25205,784.00,-75527.20365,-0.091806,2020-05-29,2020-06-03,5 days
5,959,162,191,778.83625,754.30,-23530.26375,-0.031504,2020-06-04,2020-07-15,41 days
6,-894,202,241,809.97070,694.00,103677.80580,0.143179,2020-08-03,2020-09-30,58 days


----------MIN2----------
Start                     2017-10-02 00:00:00
End                       2018-09-28 00:00:00
Duration                    361 days 00:00:00
Exposure Time [%]                   61.538462
Equity Final [$]                1102670.71555
Equity Peak [$]                  1267410.6027
Return [%]                          10.267072
Buy & Hold Return [%]              -37.130542
Return (Ann.) [%]                    9.935971
Volatility (Ann.) [%]               29.305641
Sharpe Ratio                         0.339046
Sortino Ratio                          0.5434
Calmar Ratio                         0.581311
Max. Drawdown [%]                  -17.092351
Avg. Drawdown [%]                   -6.871556
Max. Drawdown Duration      149 days 00:00:00
Avg. Drawdown Duration       42 days 00:00:00
# Trades                                   10
Win Rate [%]                             50.0
Best Trade [%]                      20.083259
Worst Trade [%]                    -11.536085
Avg. Trad

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,864,71,91,1156.69745,1389.00,200709.40320,0.200833,2018-01-09,2018-02-06,28 days
1,-920,92,94,1304.51055,1455.00,-138450.29400,-0.115361,2018-02-07,2018-02-09,2 days
2,-750,95,143,1414.96110,1258.00,117720.82500,0.110930,2018-02-12,2018-04-19,66 days
3,924,144,159,1276.28650,1201.75,-68871.72600,-0.058401,2018-04-20,2018-05-11,21 days
4,921,160,199,1205.94000,1308.00,93997.26000,0.084631,2018-05-14,2018-07-06,53 days
5,1025,208,209,1174.78655,1185.00,10468.78625,0.008694,2018-07-19,2018-07-20,1 days
6,-975,226,231,1245.80260,1242.00,3707.53500,0.003052,2018-08-14,2018-08-21,7 days
7,1041,237,251,1170.76675,1107.70,-65652.48675,-0.053868,2018-08-29,2018-09-18,20 days
8,1074,252,257,1073.28660,1045.00,-30379.80840,-0.026355,2018-09-19,2018-09-26,7 days
9,1075,258,259,1044.14305,1025.00,-20578.77875,-0.018334,2018-09-27,2018-09-28,1 days


----------MIN3----------
Start                     2020-10-02 00:00:00
End                       2021-09-30 00:00:00
Duration                    363 days 00:00:00
Exposure Time [%]                    61.47541
Equity Final [$]                 1163992.2079
Equity Peak [$]                  1197880.9517
Return [%]                          16.399221
Buy & Hold Return [%]              -46.952909
Return (Ann.) [%]                   16.980203
Volatility (Ann.) [%]               35.311921
Sharpe Ratio                         0.480863
Sortino Ratio                        0.885413
Calmar Ratio                          0.99345
Max. Drawdown [%]                  -17.092158
Avg. Drawdown [%]                   -8.132494
Max. Drawdown Duration      143 days 00:00:00
Avg. Drawdown Duration       43 days 00:00:00
# Trades                                    7
Win Rate [%]                        57.142857
Best Trade [%]                      13.573151
Worst Trade [%]                     -7.694599
Avg. Trad

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,2130,71,86,469.31165,433.20,-76917.8145,-0.076946,2021-01-18,2021-02-08,21 days
1,2176,87,91,424.08890,402.00,-48065.4464,-0.052086,2021-02-09,2021-02-16,7 days
2,2182,92,148,400.97505,455.40,118755.2409,0.135732,2021-02-17,2021-05-13,85 days
3,2447,172,189,405.99980,445.00,95433.4894,0.096060,2021-06-16,2021-07-09,23 days
4,-2593,190,228,419.91110,378.00,108675.4823,0.099809,2021-07-12,2021-09-07,57 days
5,3136,229,233,381.88100,358.15,-74420.4160,-0.062142,2021-09-08,2021-09-14,6 days
6,3071,234,243,365.80180,379.00,40531.6722,0.036080,2021-09-15,2021-09-30,15 days


----------BEST----------
Start                     2018-10-01 00:00:00
End                       2019-09-30 00:00:00
Duration                    364 days 00:00:00
Exposure Time [%]                   55.510204
Equity Final [$]                1417066.85105
Equity Peak [$]                  1422744.0068
Return [%]                          41.706685
Buy & Hold Return [%]                5.544554
Return (Ann.) [%]                   43.125091
Volatility (Ann.) [%]               32.036515
Sharpe Ratio                         1.346123
Sortino Ratio                        3.267183
Calmar Ratio                         4.204139
Max. Drawdown [%]                  -10.257771
Avg. Drawdown [%]                   -3.452101
Max. Drawdown Duration       50 days 00:00:00
Avg. Drawdown Duration       14 days 00:00:00
# Trades                                    7
Win Rate [%]                        71.428571
Best Trade [%]                      14.671331
Worst Trade [%]                     -5.704264
Avg. Trad

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,829,98,124,1205.94000,1137.15,-57026.91000,-0.057043,2019-02-21,2019-04-01,39 days
1,865,135,156,1089.36580,1179.00,77533.58300,0.082281,2019-04-16,2019-05-23,37 days
2,-869,158,184,1173.16395,1109.00,55758.47255,0.054693,2019-05-27,2019-07-02,36 days
3,965,185,208,1114.48955,1278.00,157787.58425,0.146713,2019-07-03,2019-08-06,34 days
4,-1012,209,211,1218.93625,1199.00,20175.48500,0.016355,2019-08-07,2019-08-09,2 days
5,-1016,212,235,1233.86200,1068.00,168515.79200,0.134425,2019-08-13,2019-09-13,31 days
6,1329,236,244,1070.27175,1066.00,-5677.15575,-0.003991,2019-09-17,2019-09-30,13 days


Row(id='37069', ...)